In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%env OPENAI_API_KEY=

In [3]:
# %pip install layout_prompter

In [4]:
# Local
import sys
import os

sys.path.append(os.path.abspath('../layout_prompter'))

# General configurations

In [96]:
from layout_prompter.dataset_configs import PosterLayoutDatasetConfig


dataset_config = PosterLayoutDatasetConfig()
task = "content"
input_format = "seq"
output_format = "html"
add_unk_token = False
add_index_token = True
add_sep_token = True
candidate_size = -1  # -1 represents the complete training set
num_prompt = 10

# Load raw data

In [ ]:
#from google.colab import drive
#drive.mount("/content/drive")
# base_dir = "/content/drive/My Drive/LayoutPrompter/posterlayout"

In [97]:
import os 
base_dir = os.path.join(os.getcwd(), "../", "dataset/LayoutPrompter/posterlayout")


# Process raw data

In [98]:
import os
import pandas as pd

from tqdm import tqdm
from layout_prompter.preprocessors import create_processor
from layout_prompter.utils import get_raw_data_path, read_pt, write_pt


metadata = pd.read_csv(f"{base_dir}/raw_small/train_csv_9973.csv")
processor = create_processor(dataset_config, task, metadata=metadata)

def get_processed_dataset(split):
    filename = os.path.join(
        base_dir, "dataset", dataset_config.name, "processed", task, f"{split}.pt"
    )
    if os.path.exists(filename):
        processed_data = read_pt(filename)
    else:
        processed_data = []
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        raw_path = os.path.join(base_dir, "raw_small", split, "saliencymaps_pfpn")
        raw_data = os.listdir(raw_path)
        raw_data = sorted(raw_data, key=lambda x: int(x.split("_")[0]))
        for rd in tqdm(raw_data, desc=f"{split} data processing..."):
            idx = int(rd.split("_")[0])
            data = processor(os.path.join(raw_path, rd), idx, split)
            if data:
                processed_data.append(data)
        write_pt(filename, processed_data)
    return processed_data


processed_train_dataset = get_processed_dataset("train")
processed_test_dataset = get_processed_dataset("test")

# Get target test data

In [124]:
test_idx = 20
test_data = processed_test_dataset[test_idx]

# Define dynamic exemplar selection

In [125]:
from layout_prompter.modules import create_selector


selector = create_selector(
    task=task,
    train_dataset=processed_train_dataset,
    candidate_size=candidate_size,
    num_prompt=num_prompt,
    dataset_config=dataset_config,
)

# Define input-output serialization

In [126]:
from layout_prompter.modules import create_serializer


serializer = create_serializer(
    dataset_config=dataset_config,
    task=task,
    input_format=input_format,
    output_format=output_format,
    add_index_token=add_index_token,
    add_sep_token=add_sep_token,
    add_unk_token=add_unk_token,
)

# Define GPT callar

In [127]:
from layout_prompter.parsers import GPTResponseParser
from layout_prompter.modules import GPTCallar
# from google.colab import userdata


model = "gpt-4o"
max_tokens = 800
temperature = 0.7
top_p = 1
frequency_penalty = 0.0
presence_penalty = 0.0
num_return = 10
stop_token = "\n\n"

assert os.environ["OPENAI_API_KEY"] is not None
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

parser = GPTResponseParser(
    dataset_config=dataset_config,
    output_format=output_format
)

llm = GPTCallar(
    parser=parser,
    model=model,
    max_tokens=max_tokens,
    temperature=temperature,
    top_p=top_p,
    frequency_penalty=frequency_penalty,
    presence_penalty=presence_penalty,
    num_return=num_return,
    stop_token=stop_token
)

# Define Ranker

In [128]:
from layout_prompter.modules import Ranker


ranker = Ranker()

# Define LayoutPrompter pipeline

In [129]:
from layout_prompter import LayoutPrompter


pipeline = LayoutPrompter(
    serializer=serializer,
    selector=selector,
    llm=llm,
    ranker=ranker,
)

# Generate layouts

In [ ]:
exemplars = pipeline.get_exemplars(test_data=test_data)
print(f"Exemplars: {len(exemplars)}")

# (1) Build prompt based on the exemplars
# (2) Generate layouts using LLM
# (3) Re-rank the generated results
ranked_response = pipeline(
    test_data=test_data, exemplars=exemplars
)

# Visualize the layouts

In [ ]:
from layout_prompter.visualizers import ContentAwareVisualizer, create_image_grid


canvas_path = os.path.join(base_dir, "raw_small", "test", "image_canvas")
visualizer = ContentAwareVisualizer(dataset_config=dataset_config, canvas_path=canvas_path)
images = visualizer(ranked_response, test_idx=test_idx)

create_image_grid(images)